In [ ]:
import torch
import numpy as np
import torch.nn.functional as F
import torch.nn as A
import pandas as pd
from sklearn.manifold import TSNE
import sklearn.manifold 
from matplotlib import pyplot as plt
import re
import codecs
from numpy  import array
from scipy import stats
# from scipy.spatial import distance
from sklearn.metrics.pairwise import cosine_similarity
import os
from tensorboardX import SummaryWriter
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader

from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics.pairwise import cosine_similarity

# PREPROCESSING

In [ ]:
from preprocessing import Preprocess
preprocess = Preprocess()

# TRAINING

In [ ]:
from dataset import LoadedDataSet

In [ ]:
from word2vec import Word2Vec

In [ ]:
dimensionSize = 8
num_epochs = 20
lr = 0.001
batchSize = 80

model = Word2Vec(preprocess.vocabulary_size, dimensionSize)
optimizer = torch.optim.Adam(model.parameters(), lr)

# print('total amount of batches {}'.format(len(idx_pairs) / batchSize))
for e in range(num_epochs):
#     fasdfsasdfadsdsfdsaf
    dataset = LoadedDataSet(preprocess.idx_pairs)
    loader = DataLoader(dataset, batchSize, shuffle=True)    
    pbar = tqdm(loader)

    for data, target in pbar:           
#         compute gradient and do optimizer step
        optimizer.zero_grad()
        loss = model.forward(data, target)  
        loss.backward()
#       think about adding clipping here
        optimizer.step()       
        pbar.set_postfix(loss=loss.item())    
    

In [ ]:
if not os.path.exists("SOFTMAX_trained"):
    os.makedirs("SOFTMAX_trained")

In [ ]:
torch.save(outputLayer, os.path.join("SOFTMAX_trained", "test#2.w2v"))

# EVALUATING TRAINED MODEL

In [ ]:
loadedLayer = torch.load(os.path.join("SOFTMAX_trained", "test#2.w2v"))

In [ ]:
# extracts the 508 Annotator Results as the Gold-Standard
corpus_filename = '../data/EmoSim508.json'
corpus = open(corpus_filename).read()
annotator_similarity_score_508 = list(array(re.findall('(?<=_Annotator_Agreement": )(.*?)(?=\})', corpus)))

# extract Wijeratne's Cosine_Similarities of the model which was trained on Google_Sense_Labels
google_sense_labels_score_508 = list(array(re.findall('(?<=Google_Sense_Label": )(.*?)(?=\,)', corpus)))

# glyph_pairs_1016
unicode_pairs_1016 = re.findall('(?<=unicodelong": "\\\)(.*?)(?=")', corpus)    
glyph_pairs_1016 = [codecs.decode(unicode_pairs_1016[x].replace(str('\\\\'),str('\\')).replace('_',''), 'unicode_escape') for x in range(len(unicode_pairs_1016))]

# computation of Cosine Similarity
goldstandard = []
selftrained = []
google_sense_labels = []
for x in range(len(annotator_similarity_score_508)):
    cosineSimilarity = None
    
    emoji1 = glyph_pairs_1016.pop(0)
    emoji2 = glyph_pairs_1016.pop(0)
    
    try:
        cosineSimilarity = cosine_similarity(loadedLayer.detach().cpu().numpy()[word2idx[emoji1]].reshape(-1,dimensionSize), loadedLayer.detach().cpu().numpy()[word2idx[emoji2]].reshape(-1,dimensionSize))[0][0]
    except:
        print('the cosine similarity between ' + emoji1 + ' and ' + emoji2 + ' could not be computed.')
    
    if(cosineSimilarity is not None):
        goldstandard.append(annotator_similarity_score_508.pop(0))
        selftrained.append(cosineSimilarity)
        google_sense_labels.append(float(google_sense_labels_score_508.pop(0)))
        

# skalierter GoldStandard
min_max_scaler = preprocessing.MinMaxScaler()
scaled_goldstandard = min_max_scaler.fit_transform(np.asarray(goldstandard).reshape(-1, 1))

print()

# computation of SPEARRANK CORRELATION COEFFICIENT
meinSPEARMAN = stats.spearmanr(goldstandard, selftrained)
seinSPEARMAN = stats.spearmanr(goldstandard, google_sense_labels)
print('mein Spearman: {}'.format(meinSPEARMAN.correlation))
print('sein Spearman: {}'.format(seinSPEARMAN.correlation))


# computation of MAE
meinMAE = mean_absolute_error(scaled_goldstandard, min_max_scaler.fit_transform(np.asarray(selftrained).reshape(-1, 1)))
seinMAE = mean_absolute_error(scaled_goldstandard, google_sense_labels)
print('mein MAE ist {}'.format(meinMAE))
print('sein MAE ist {}'.format(seinMAE))


# computation of MSE
meinMSE = mean_squared_error(scaled_goldstandard, min_max_scaler.fit_transform(np.asarray(selftrained).reshape(-1, 1)))
seinMSE = mean_squared_error(scaled_goldstandard, google_sense_labels)
print('mein MSE ist {}'.format(meinMSE))
print('sein MSE ist {}'.format(seinMSE))
